In [1]:
from keras.callbacks import ModelCheckpoint
from keras.models import Model, load_model, Sequential, Sequential
from keras.layers import Dense, Activation, Dropout, Input, Masking, TimeDistributed, LSTM, Conv1D
from keras.layers import GRU, Bidirectional, BatchNormalization, Reshape
from keras.utils import Sequence
import numpy as np

Using TensorFlow backend.


In [2]:
n_freq = 101
Tx = 1332

In [3]:
def model(input_shape):
    """
    Function creating the model's graph in Keras.
    
    Argument:
    input_shape -- shape of the model's input data (using Keras conventions)

    Returns:
    model -- Keras model instance
    """
    
    X_input = Input(shape = input_shape)
       
    # Step 1: CONV layer (≈4 lines)
    X = Conv1D(196,15,strides=4)(X_input)                         
    X = BatchNormalization()(X)                           
    X = Activation('relu')(X)              
    X = Dropout(0.8)(X)                      
    
    # Step 2: First GRU Layer (≈4 lines)
    X = GRU(units = 128, return_sequences = True)(X)                              
    X = Dropout(0.8)(X)                                
    X = BatchNormalization()(X)                                 
    
    # Step 3: Second GRU Layer (≈4 lines)
    X = GRU(units = 128, return_sequences = True)(X)                                
    X = Dropout(0.8)(X)                                
    X = BatchNormalization()(X)
    X = Dropout(0.8)(X)                               
    
    # Step 4: Time-distributed dense layer (≈1 line)
    X = TimeDistributed(Dense(1, activation = "sigmoid"))(X) # time distributed  (sigmoid)

    model = Model(inputs = X_input, outputs = X)
    
    return model 

In [4]:
model = model(input_shape = (Tx, n_freq))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1332, 101)         0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 330, 196)          297136    
_________________________________________________________________
batch_normalization_1 (Batch (None, 330, 196)          784       
_________________________________________________________________
activation_1 (Activation)    (None, 330, 196)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 330, 196)          0         
_________________________________________________________________
gru_1 (GRU)                  (None, 330, 128)          124800    
_________________________________________________________________
dropout_2 (Dropout)          (None, 330, 128)          0         
__________

In [5]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=["accuracy"])

In [8]:
x = np.load(f"./_data/x{1}.npy")
y = np.load(f"./_data/y{1}.npy")

for i in range(2,26):
    x_ = np.load(f"./_data/x{i}.npy")
    y_ = np.load(f"./_data/y{i}.npy")
    x = np.append(x, x_, axis=0)
    y = np.append(y, y_, axis=0)
    


In [9]:
print(x.shape)
print(y.shape)
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33)

(2500, 1332, 101)
(2500, 330, 1)


In [10]:
network = model
checkpointer = ModelCheckpoint(filepath='./model_{}.weights.best.hdf5'.format(network), verbose=1,save_best_only=True)

In [11]:
history = model.fit(x=x, y=y, epochs=16, batch_size=48, validation_split=0.33,callbacks=[checkpointer],shuffle=True)

Train on 1674 samples, validate on 826 samples
Epoch 1/16
1674/1674 [==============================] - 58s 34ms/step - loss: 1.0566 - acc: 0.6451 - val_loss: 0.5345 - val_acc: 0.9700

Epoch 00001: val_loss improved from inf to 0.53453, saving model to ./model_<keras.engine.training.Model object at 0x7fbc592777f0>.weights.best.hdf5
Epoch 2/16
1674/1674 [==============================] - 36s 21ms/step - loss: 0.7725 - acc: 0.8091 - val_loss: 0.4418 - val_acc: 0.9770

Epoch 00002: val_loss improved from 0.53453 to 0.44179, saving model to ./model_<keras.engine.training.Model object at 0x7fbc592777f0>.weights.best.hdf5
Epoch 3/16
1674/1674 [==============================] - 36s 22ms/step - loss: 0.6195 - acc: 0.8423 - val_loss: 0.3304 - val_acc: 0.9885

Epoch 00003: val_loss improved from 0.44179 to 0.33036, saving model to ./model_<keras.engine.training.Model object at 0x7fbc592777f0>.weights.best.hdf5
Epoch 4/16
1674/1674 [==============================] - 35s 21ms/step - loss: 0.5058 - 

In [36]:
val_loss = history.history['val_loss']
loss = history.history['loss']

In [35]:
import matplotlib.pyplot as plt
plt.plot(range(16),val_loss)
plt.plot(range(16),loss)
plt.title("Loss vs No. of Epochs")
plt.legend(["validation loss","training loss"])
plt.xlabel('No. of Epochs')
plt.ylabel('Loss')
ax = plt.show()
plt.savefig("Loss")